# CLIP Embedding 훔쳐오기

In [ ]:
import torch
import clip
from PIL import Image

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
image = preprocess(Image.open("CLIP.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["diagram a", "dog a", "a cat", "a a"]).to(device)

In [ ]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

In [ ]:
feature = model.encode_image(image)
feature.shape

In [ ]:
feature = model.visual.transformer.register_forward_hook(get_features('feats'))

In [ ]:
print(feature)

In [ ]:
features = {}
model.encode_image(image)

In [ ]:
features['feats'].permute(1, 0, 2).shape

In [ ]:
features = {}
model.transformer.register_forward_hook(get_features('text_emb'))

In [ ]:
text_features = model.encode_text(text)
print(text_features.shape)

In [ ]:
features['text_emb'].permute(1,0,2).shape

In [ ]:
features['text_emb'][:4,0,:]

In [ ]:
features['text_emb'][:4,1,:]

In [ ]:
features['text_emb'][:4,2,:]

In [ ]:
features['text_emb'][:4,3,:]

# GPT2 Embedding Matrix 꺼내오기

In [29]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token="<|startoftext|>")
model = GPT2LMHeadModel.from_pretrained('gpt2')
text = ["<|startoftext|> Replace me by any text you'd like. <|endoftext|>", "hi my name"]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
token = tokenizer(text, return_tensors='pt')
print(token)

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [3]:
wte = model.transformer.wte
wpe = model.transformer.wpe

In [4]:
input_ids = token['input_ids']
input_shape = input_ids.shape
past_length = 0

In [5]:
position_ids = torch.arange(past_length, input_shape[-1] + past_length)
position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])
print(position_ids.shape)

NameError: name 'torch' is not defined

In [7]:
input_embedding = wte(input_ids)
#position_embedding = wpe(position_ids)
#text_embedding = input_embedding + position_embedding
print(input_embedding.shape)
#print(position_embedding.shape)

torch.Size([1, 10, 768])


In [26]:
def create_mask_per_text(num_sentence, num_words, num_preceding_image, num_embedds):
    mask = torch.ones((num_words, num_sentence * num_embedds))
    mask[:, :num_preceding_image * num_embedds] = 0
    return mask

In [27]:
import torch
mask = create_mask_per_sentence(2,3,1,50)
print(mask.shape)
print(mask)

torch.Size([3, 100])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [28]:
mask = torch.Tensor([])
num_embedds = 50
texts = ["hi, my name is", "cat a boy"]
for text in texts:
    mask_per_text = create_mask_per_text(text, num_embedds)
    mask = torch.cat([mask, mask_per_text], dim=-2)

TypeError: create_mask_per_text() missing 2 required positional arguments: 'num_preceding_image' and 'num_embedds'